In [7]:
import json
import numpy as np

N_STATES = 6
N_OBS = 6
EPS = 1e-12

In [8]:
A_mask = np.array([
# to:  0 1 2 3 4 5
    [1,1,1,1,1,1],  # from 0 normal → all
    [1,1,1,0,0,0],  # from 1 happy → 0,1,2
    [1,0,0,0,0,1],  # from 2 very happy → 0,5
    [1,0,0,1,1,0],  # from 3 sad → 0,3,4
    [1,0,0,0,0,1],  # from 4 very sad → 0,5
    [1,0,0,0,0,1],  # from 5 not available → 0,5
])


In [9]:
def load_data(path):
    with open(path, "r") as f:
        return json.load(f)
def normalize_masked(row, mask):
    row = row * mask
    s = np.sum(row)
    if s == 0:
        row = mask / np.sum(mask)
    else:
        row = row / s
    return row


In [10]:
def forward(obs, pi, A, B):
    T = len(obs)
    alpha = np.zeros((T, N_STATES))
    scale = np.zeros(T)

    alpha[0] = pi * B[:, obs[0]]
    scale[0] = np.sum(alpha[0]) + EPS
    alpha[0] /= scale[0]

    for t in range(1, T):
        for j in range(N_STATES):
            alpha[t, j] = B[j, obs[t]] * np.sum(alpha[t-1] * A[:, j])
        scale[t] = np.sum(alpha[t]) + EPS
        alpha[t] /= scale[t]

    log_prob = np.sum(np.log(scale))
    return alpha, scale, log_prob


In [11]:
def backward(obs, A, B, scale):
    T = len(obs)
    beta = np.zeros((T, N_STATES))
    beta[-1] = 1.0 / scale[-1]

    for t in reversed(range(T-1)):
        for i in range(N_STATES):
            beta[t, i] = np.sum(A[i] * B[:, obs[t+1]] * beta[t+1])
        beta[t] /= scale[t]

    return beta


In [12]:
def baum_welch(sequences, A_mask, n_iter=10):

    # Initial probabilities
    pi = np.ones(N_STATES) / N_STATES

    # Initialize A respecting mask
    A = np.zeros((N_STATES, N_STATES))
    for i in range(N_STATES):
        A[i] = normalize_masked(np.random.rand(N_STATES), A_mask[i])

    # Initialize B
    B = np.array([np.ones(N_OBS) / N_OBS for _ in range(N_STATES)])

    for _ in range(n_iter):

        #print(_)
        pi_new = np.zeros(N_STATES)
        A_new = np.zeros((N_STATES, N_STATES))
        B_new = np.zeros((N_STATES, N_OBS))

        #x=0
        for obs in sequences:
            alpha, scale, _ = forward(obs, pi, A, B)
            beta = backward(obs, A, B, scale)
            T = len(obs)

            gamma = alpha * beta
            gamma /= np.sum(gamma, axis=1, keepdims=True)

            pi_new += gamma[0]

            for t in range(T-1):
                denom = np.sum(
                    alpha[t][:, None] * A * B[:, obs[t+1]] * beta[t+1]
                ) + EPS

                for i in range(N_STATES):
                    for j in range(N_STATES):
                        if A_mask[i, j] == 1:
                            A_new[i, j] += (
                                alpha[t, i] * A[i, j] *
                                B[j, obs[t+1]] * beta[t+1, j]
                            ) / denom
            #x+=1
            #print(x)
            for t in range(T):
                B_new[:, obs[t]] += gamma[t]

        pi = pi_new / np.sum(pi_new)

        for i in range(N_STATES):
            A[i] = normalize_masked(A_new[i], A_mask[i])

        for i in range(N_STATES):
            B[i] = B_new[i] / (np.sum(B_new[i]) + EPS)

    return pi, A, B


In [ ]:
#from google.colab import files

#uploaded = files.upload()  # choose train.json

#text_path = list(uploaded.keys())[0]  # this will be "train.json"
#after uploading we have :
text_path=load_data("train.json")
train_sequences = load_data(text_path)
pi, A, B = baum_welch(train_sequences, A_mask)

print(pi,A,B)

Streaming output truncated to the last 5000 lines.
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203

In [5]:
np.savez("hmm_parameters_2.npz", pi=pi, A=A, B=B)

with open("hmm_parameters_2.txt", "w") as f:
    f.write("Initial probabilities π:\n")
    for i, p in enumerate(pi):
        f.write(f"π(State ({i})) = {p}\n")

    f.write("\nTransition matrix A:\n")
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            f.write(f"A(State ({i}) → State ({j})) = {A[i, j]}\n")

    f.write("\nEmission matrix B:\n")
    for i in range(B.shape[0]):
        for k in range(B.shape[1]):
            f.write(f"B(State ({i}), Observation ({k})) = {B[i, k]}\n")


# b) calculating the log-probabilities of each sequence in the test data

In [15]:
#from google.colab import files

#uploaded = files.upload()  # Choose hafez.txt

#test_sequences= list(uploaded.keys())[0]

params = np.load("hmm_parameters_2.npz")
pi, A, B = params["pi"], params["A"], params["B"]

test_sequences = load_data("test.json")

log_probs = []
for obs in test_sequences:
    _, _, logp = forward(obs, pi, A, B)
    log_probs.append(logp)

with open("test_log_probs.json", "w") as f:
    json.dump(log_probs, f)

print(log_probs)


[np.float64(-178.98115444051916), np.float64(-179.08452276873322), np.float64(-179.16833016088918), np.float64(-179.07088633610516), np.float64(-179.47443801015473), np.float64(-179.04978960051102), np.float64(-179.2919201590637), np.float64(-179.0948212458615), np.float64(-179.09594643989283), np.float64(-179.31481808726156), np.float64(-179.14053376815386), np.float64(-179.18939701571688), np.float64(-179.1416089311766), np.float64(-179.08666353025842), np.float64(-178.77539969000637), np.float64(-179.37487110867076), np.float64(-179.46991594499423), np.float64(-179.27495108919362), np.float64(-179.13432734972923), np.float64(-179.1264725210452), np.float64(-179.04072936289927), np.float64(-179.44812220556872), np.float64(-179.22728326388096), np.float64(-179.19259806278734), np.float64(-179.0895142355233), np.float64(-178.79382176195307), np.float64(-179.06593500349945), np.float64(-179.2069139384501), np.float64(-179.14125643400388), np.float64(-179.0710022864736), np.float64(-179.

# c) viterbi algorithm  to find the most probable states in each test sample

In [20]:
def viterbi(obs, pi, A, B, A_mask):
    T = len(obs)
    delta = np.full((T, N_STATES), -np.inf)
    psi = np.zeros((T, N_STATES), dtype=int)

    delta[0] = np.log(pi + EPS) + np.log(B[:, obs[0]] + EPS)

    for t in range(1, T):
        for j in range(N_STATES):
            candidates = []
            for i in range(N_STATES):
                if A_mask[i, j] == 1:
                    candidates.append(delta[t-1, i] + np.log(A[i, j] + EPS))
                else:
                    candidates.append(-np.inf)

            candidates = np.array(candidates)
            psi[t, j] = np.argmax(candidates)
            delta[t, j] = np.max(candidates) + np.log(B[j, obs[t]] + EPS)

    states = np.zeros(T, dtype=int)
    states[-1] = np.argmax(delta[-1])

    for t in reversed(range(T-1)):
        states[t] = psi[t+1, states[t+1]]

    return states.tolist()


In [21]:
#saving in a json format
viterbi_results = []

for obs in test_sequences:
    path = viterbi(obs, pi, A, B, A_mask)
    viterbi_results.append(path)

with open("viterbi_states.json", "w") as f:
    json.dump(viterbi_results, f)


In [22]:
#saving in a text format
viterbi_results = []
state_names = {
    0: "normal",
    1: "happy",
    2: "very_happy",
    3: "sad",
    4: "very_sad",
    5: "not_available"
}

with open("viterbi_states.txt", "w") as f:
    for seq_idx, obs in enumerate(test_sequences):

        path = viterbi(obs, pi, A, B, A_mask)
        viterbi_results.append(path)

        f.write(f"Sequence {seq_idx}\n")
        f.write("Time\tObservation\tHidden_State\n")

        for t in range(len(obs)):
            o = obs[t]
            s = path[t]
            s_name = state_names[s]

            f.write(f"{t}\t{o}\t\t{s_name}\n")

        f.write("\n" + "-"*40 + "\n\n")
